In [1]:
import sys
if ".." not in sys.path:
    sys.path.append("..")

import tensorflow as tf
from tensorflow import keras

from src.constants import LEN_UCI_MOVES

activation = "relu"
inputs = keras.Input(shape=(70), name="all_input")
board_input = keras.layers.Lambda(lambda x: tf.slice(x, (0, 0), (-1, 64)), name="board_input")(inputs)
misc_input = keras.layers.Lambda(lambda x: tf.slice(x, (0, 64), (-1, 6)), name="misc_input")(inputs)

conv3_model = tf.keras.models.Sequential([  # 1:65
    keras.layers.Reshape((8, 8, 1), input_shape=(64,)),
    keras.layers.Conv2D(32, 3, padding="same", activation=activation),
    keras.layers.Conv2D(32, 3, padding="same", activation=activation),
    keras.layers.Flatten(),
    keras.layers.Dense(64)
], name="conv_model_k3")
conv3_output = conv3_model(board_input)

conv5_model = tf.keras.models.Sequential([  # 1:65
    keras.layers.Reshape((8, 8, 1), input_shape=(64,)),
    keras.layers.Conv2D(32, 5, padding="same", activation=activation),
    keras.layers.Conv2D(32, 5, padding="same", activation=activation),
    keras.layers.Flatten(),
    keras.layers.Dense(64)
], name="conv_model_k5")
conv5_output = conv5_model(board_input)

conv7_model = tf.keras.models.Sequential([  # 1:65
    keras.layers.Reshape((8, 8, 1), input_shape=(64,)),
    keras.layers.Conv2D(32, 7, padding="same", activation=activation),
    keras.layers.Conv2D(32, 7, padding="same", activation=activation),
    keras.layers.Flatten(),
    keras.layers.Dense(64)
], name="conv_model_k7")
conv7_output = conv7_model(board_input)

dense_inputs = keras.layers.concatenate([conv3_output, conv5_output, conv7_output, misc_input])
dense_model = tf.keras.models.Sequential([  # tf.concat([input[:1], conv_model_output, input[65:]], axis=0)
    keras.layers.Input(198),
    keras.layers.Dense(256, activation=activation),
    keras.layers.Dense(256, activation=activation),
    keras.layers.Dense(1, activation="sigmoid")
], name="dense_model")
dense_output = dense_model(dense_inputs)

model = keras.models.Model(inputs, dense_output, name="conv2d_357")
model.compile(
    optimizer = keras.optimizers.legacy.Adam(learning_rate=0.001),
    loss = keras.losses.BinaryCrossentropy(),
    metrics = [keras.metrics.BinaryCrossentropy()],
)

In [2]:
import pandas as pd
import numpy as np

test_file = "../data/split/processed/chess1.pkl"
df = pd.read_pickle(test_file)
X = np.concatenate([np.array(df["obs_board"].values.tolist()), np.array(df["obs_misc"].values.tolist())], axis=1)
Y = df["outcome"].values

In [3]:
model.fit(X, Y)

31250/31250 [==============================] - 194s 6ms/step - loss: 0.6160 - binary_crossentropy: 0.6160


In [4]:
predictions = model.predict(X)

31250/31250 [==============================] - 61s 2ms/step


In [10]:
accuracy = tf.keras.metrics.BinaryAccuracy(
    name='binary_accuracy', dtype=None, threshold=0.5
)
accuracy.update_state(Y, predictions)
accuracy.result().numpy()

0.585358

In [11]:
cross_entropy = tf.keras.metrics.BinaryCrossentropy(
    name='binary_crossentropy',
    dtype=None,
    from_logits=False,
    label_smoothing=0
)
cross_entropy.update_state(Y, predictions)
cross_entropy.result().numpy()

0.5996955